In [1]:
import os
while not os.path.exists('.gitignore'):
	os.chdir(os.path.abspath(os.path.join('../')))
from common_imports import *

Defaulting to user installation because normal site-packages is not writeable


In [2]:
df = pd.read_parquet('_data/raw/juros-bancarios/combined.parquet')

In [3]:
df

,Segmento,Modalidade,InstituicaoFinanceira,TaxaJurosAoAno,data
0,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS DE MERCADO...,CPCI,"10,54",2012-01-01
1,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS DE MERCADO...,BCO CITIBANK S.A.,"11,66",2012-01-01
2,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS DE MERCADO...,BCO DO BRASIL S.A.,"16,40",2012-01-01
3,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS DE MERCADO...,BRAZILIAN MORTGAGES CH S.A.,"18,50",2012-01-01
4,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS DE MERCADO...,BANCO INTER,"31,13",2012-01-01
...,...,...,...,...,...
2406019,Pessoa Jurídica,Conta garantida - Pré-fixado,BCO PINE S.A.,"145,84",2024-09-17
2406020,Pessoa Jurídica,Conta garantida - Pré-fixado,BCO GUANABARA S.A.,"185,13",2024-09-17
2406021,Pessoa Jurídica,Conta garantida - Pré-fixado,BCO SOFISA S.A.,"227,68",2024-09-17
2406022,Pessoa Jurídica,Conta garantida - Pré-fixado,BCO FIBRA S.A.,"388,00",2024-09-17


In [4]:
df['TaxaJurosAoAno'] = df['TaxaJurosAoAno'].str.replace(',', '.').astype(float)

In [5]:
df['InstituicaoFinanceira'].value_counts()

InstituicaoFinanceira
BCO BRADESCO S.A.                 68759
BCO DO BRASIL S.A.                68634
BCO SANTANDER (BRASIL) S.A.       66638
BCO DO ESTADO DO RS S.A.          63121
ITAÚ UNIBANCO S.A.                59172
                                  ...  
MERCANTIL BRASIL LEASING SA AM        8
BMG LEASING S/A-AM                    5
BANIF INVESTIMENTO                    5
GOLDMAN SACHS DO BRASIL BM S.A        5
TRADIÇÃO                              5
Name: count, Length: 270, dtype: int64

In [6]:
df.head()

,Segmento,Modalidade,InstituicaoFinanceira,TaxaJurosAoAno,data
0,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS DE MERCADO...,CPCI,10.54,2012-01-01
1,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS DE MERCADO...,BCO CITIBANK S.A.,11.66,2012-01-01
2,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS DE MERCADO...,BCO DO BRASIL S.A.,16.40,2012-01-01
3,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS DE MERCADO...,BRAZILIAN MORTGAGES CH S.A.,18.50,2012-01-01
4,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS DE MERCADO...,BANCO INTER,31.13,2012-01-01


In [7]:
df['Modalidade'].value_counts()

Modalidade
Crédito pessoal não-consignado - Pré-fixado                                                    222594
Cheque especial - Pré-fixado                                                                   168299
Crédito pessoal consignado privado - Pré-fixado                                                154784
Desconto de duplicatas - Pré-fixado                                                            152514
Capital de giro com prazo até 365 dias - Pré-fixado                                            142902
Aquisição de veículos - Pré-fixado                                                             140624
Cartão de crédito - rotativo total - Pré-fixado                                                132475
Capital de giro com prazo superior a 365 dias - Pré-fixado                                     126726
Crédito pessoal consignado público - Pré-fixado                                                125525
Aquisição de outros bens - Pré-fixado                                  

In [8]:
df['SubClassificacao'] = df['Modalidade'].apply(lambda x: x.split(' - ')[-1])
df['Modalidade'] = df['Modalidade'].apply(lambda x: x.rsplit(' - ', 1)[0])

In [9]:
df['Modalidade'].value_counts()

Modalidade
Capital de giro com prazo até 365 dias            229886
Crédito pessoal não-consignado                    222594
Capital de giro com prazo superior a 365 dias     216567
Conta garantida                                   216502
Cheque especial                                   168299
Crédito pessoal consignado privado                154784
Desconto de duplicatas                            152514
Aquisição de veículos                             140624
Cartão de crédito - rotativo total                132475
Crédito pessoal consignado público                125525
Aquisição de outros bens                          124514
Cartão de crédito - parcelado                     120316
Crédito pessoal consignado INSS                   114840
Adiantamento sobre contratos de câmbio (ACC)      106436
Desconto de cheques                                95039
Vendor                                             40170
Antecipação de faturas de cartão de crédito        28305
Arrendamento mercant

In [10]:
df['SubClassificacao'].value_counts()

SubClassificacao
Pré-fixado                                      2010994
Pós-fixado referenciado em juros flutuantes      283618
Pós-fixado referenciado em moeda estrangeira     106436
PÓS-FIXADO REFERENCIADO EM TR                      3521
PRÉ-FIXADO                                          927
PÓS-FIXADO REFERENCIADO EM IPCA                     528
Name: count, dtype: int64

In [11]:
def transform_taxa(valor_taxa, periodo_desejado, periodo_atual):
	vt = Decimal(valor_taxa)
	pd = Decimal(periodo_desejado)
	pa = Decimal(periodo_atual)
 
	a = 1 + (vt / 100)
	b = pd / pa
	c = a ** b
	d = (c - 1) * 100

	return d

In [12]:
def trim_decimal(n):
	valor = Decimal(str(n).rstrip('0').rstrip('.'))
	return valor.quantize(Decimal('1.0000'), rounding=ROUND_HALF_UP)

In [13]:
df

,Segmento,Modalidade,InstituicaoFinanceira,TaxaJurosAoAno,data,SubClassificacao
0,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS DE MERCADO,CPCI,10.54,2012-01-01,PRÉ-FIXADO
1,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS DE MERCADO,BCO CITIBANK S.A.,11.66,2012-01-01,PRÉ-FIXADO
2,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS DE MERCADO,BCO DO BRASIL S.A.,16.40,2012-01-01,PRÉ-FIXADO
3,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS DE MERCADO,BRAZILIAN MORTGAGES CH S.A.,18.50,2012-01-01,PRÉ-FIXADO
4,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS DE MERCADO,BANCO INTER,31.13,2012-01-01,PRÉ-FIXADO
...,...,...,...,...,...,...
2406019,Pessoa Jurídica,Conta garantida,BCO PINE S.A.,145.84,2024-09-17,Pré-fixado
2406020,Pessoa Jurídica,Conta garantida,BCO GUANABARA S.A.,185.13,2024-09-17,Pré-fixado
2406021,Pessoa Jurídica,Conta garantida,BCO SOFISA S.A.,227.68,2024-09-17,Pré-fixado
2406022,Pessoa Jurídica,Conta garantida,BCO FIBRA S.A.,388.00,2024-09-17,Pré-fixado


In [14]:
df['TaxaJurosAoMes'] = df.apply(lambda x: trim_decimal(transform_taxa(x['TaxaJurosAoAno'], 1, 12)), axis=1).astype(float)

In [15]:
SUBCLASSIFICACAO_REPLACE_MAP = {
	'PÓS-FIXADO': 'Pós-fixado',
	'PRÉ-FIXADO': 'Pré-fixado',
	'PÓS-FIXADO REFERENCIADO EM IPCA': 'Pós-fixado referenciado em IPCA',
	'PÓS-FIXADO REFERENCIADO EM TR': 'Pós-fixado referenciado em TR',
}
df['SubClassificacao'] = df['SubClassificacao'].replace(SUBCLASSIFICACAO_REPLACE_MAP)

In [16]:
df['SubClassificacao'].value_counts()

SubClassificacao
Pré-fixado                                      2011921
Pós-fixado referenciado em juros flutuantes      283618
Pós-fixado referenciado em moeda estrangeira     106436
Pós-fixado referenciado em TR                      3521
Pós-fixado referenciado em IPCA                     528
Name: count, dtype: int64

In [17]:
MODALIDADE_REPLACE_MAP = {
	'FINANCIAMENTO IMOBILIÁRIO COM TAXAS DE MERCADO': 'Financiamento imobiliário com taxas de mercado',
	'FINANCIAMENTO IMOBILIÁRIO COM TAXAS REGULADAS': 'Financiamento imobiliário com taxas reguladas',
}
df['Modalidade'] = df['Modalidade'].replace(MODALIDADE_REPLACE_MAP)

In [18]:
df['Modalidade'].value_counts()

Modalidade
Capital de giro com prazo até 365 dias            229886
Crédito pessoal não-consignado                    222594
Capital de giro com prazo superior a 365 dias     216567
Conta garantida                                   216502
Cheque especial                                   168299
Crédito pessoal consignado privado                154784
Desconto de duplicatas                            152514
Aquisição de veículos                             140624
Cartão de crédito - rotativo total                132475
Crédito pessoal consignado público                125525
Aquisição de outros bens                          124514
Cartão de crédito - parcelado                     120316
Crédito pessoal consignado INSS                   114840
Adiantamento sobre contratos de câmbio (ACC)      106436
Desconto de cheques                                95039
Vendor                                             40170
Antecipação de faturas de cartão de crédito        28305
Arrendamento mercant

In [19]:
MODALIDADE_REPLACE_MAP = {
	'Pessoa Física': 'PF',
	'Pessoa Jurídica': 'PJ',
}
df['Segmento'] = df['Segmento'].replace(MODALIDADE_REPLACE_MAP)

In [20]:
df['Segmento'].value_counts()

Segmento
PF    1277454
PJ    1128570
Name: count, dtype: int64

In [21]:
df['InstituicaoFinanceira'].value_counts()

InstituicaoFinanceira
BCO BRADESCO S.A.                 68759
BCO DO BRASIL S.A.                68634
BCO SANTANDER (BRASIL) S.A.       66638
BCO DO ESTADO DO RS S.A.          63121
ITAÚ UNIBANCO S.A.                59172
                                  ...  
MERCANTIL BRASIL LEASING SA AM        8
BMG LEASING S/A-AM                    5
BANIF INVESTIMENTO                    5
GOLDMAN SACHS DO BRASIL BM S.A        5
TRADIÇÃO                              5
Name: count, Length: 270, dtype: int64

In [22]:
df.dtypes

Segmento                  object
Modalidade                object
InstituicaoFinanceira     object
TaxaJurosAoAno           float64
data                      object
SubClassificacao          object
TaxaJurosAoMes           float64
dtype: object

In [23]:
df.rename(columns={
    'Segmento': 'ct_classificacao',
    'Modalidade': 'ct_modalidade',
    'InstituicaoFinanceira': 'ct_instituicao_financeira',
    'SubClassificacao': 'ct_indexador_modalidade',
    'TaxaJurosAoAno': 'vl_taxa_juros_ano',
    'TaxaJurosAoMes': 'vl_taxa_juros_mes',
}, inplace=True)

In [24]:
df.dtypes

ct_classificacao              object
ct_modalidade                 object
ct_instituicao_financeira     object
vl_taxa_juros_ano            float64
data                          object
ct_indexador_modalidade       object
vl_taxa_juros_mes            float64
dtype: object